## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-10-01-14-36 +0000,nyt,"With Qatar Attack, Israel Again Leaves Trump i...",https://www.nytimes.com/2025/09/09/us/politics...
1,2025-09-10-01-09-20 +0000,startribune,Service workers across the University of Minne...,https://www.startribune.com/service-workers-ac...
2,2025-09-10-01-02-56 +0000,nyt,"Oliver North and Fawn Hall, Key Figures in Ira...",https://www.nytimes.com/2025/09/09/us/oliver-n...
3,2025-09-10-01-02-31 +0000,nyt,"Boston Mayor Advances to General Election, Wil...",https://www.nytimes.com/2025/09/09/us/boston-m...
4,2025-09-10-01-01-06 +0000,nyt,Israel Attempts to Kill Hamas Leadership in Ai...,https://www.nytimes.com/2025/09/09/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2362/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,49
282,new,24
80,nepal,18
2,israel,17
229,court,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
72,2025-09-09-22-19-57 +0000,wapo,Israeli strike targeting Hamas in Qatar ‘does ...,https://www.washingtonpost.com/world/2025/09/0...,128
302,2025-09-09-09-30-00 +0000,wsj,When a federal appeals court recently rejected...,https://www.wsj.com/politics/policy/cases-on-t...,103
179,2025-09-09-17-44-10 +0000,nypost,White House fumes over Epstein birthday book s...,https://nypost.com/2025/09/09/us-news/wh-fumes...,99
9,2025-09-10-00-52-21 +0000,bbc,Trump says strike on Doha 'does not advance Is...,https://www.bbc.com/news/articles/cx25711r8jxo...,94
54,2025-09-09-22-48-16 +0000,nypost,Supreme Court allows Trump to temporarily with...,https://nypost.com/2025/09/09/us-news/supreme-...,92


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
72,128,2025-09-09-22-19-57 +0000,wapo,Israeli strike targeting Hamas in Qatar ‘does ...,https://www.washingtonpost.com/world/2025/09/0...
169,83,2025-09-09-18-28-45 +0000,nypost,French President Emmanuel Macron names loyalis...,https://nypost.com/2025/09/09/world-news/frenc...
192,77,2025-09-09-17-00-25 +0000,nyt,Protests Against Nepal’s Social Media Ban Show...,https://www.nytimes.com/2025/09/09/world/asia/...
323,53,2025-09-09-07-10-42 +0000,nypost,Bill Clinton letter in Epstein ‘birthday book’...,https://nypost.com/2025/09/09/us-news/bill-cli...
234,50,2025-09-09-14-49-29 +0000,nypost,Greta Thunberg’s Gaza flotilla was not hit by ...,https://nypost.com/2025/09/09/world-news/greta...
281,47,2025-09-09-10-36-47 +0000,nypost,‘Brutally savage’ Russian airstrike kills 24 p...,https://nypost.com/2025/09/09/world-news/russi...
325,46,2025-09-09-06-41-09 +0000,bbc,Thai court rules ex-PM Thaksin must serve one ...,https://www.bbc.com/news/articles/cly7k2g37g4o...
191,43,2025-09-09-17-05-42 +0000,nypost,"US test scores hit damning new lows in math, r...",https://nypost.com/2025/09/09/us-news/us-test-...
154,37,2025-09-09-19-29-47 +0000,nypost,Sicko accused of brutally stabbing Ukrainian r...,https://nypost.com/2025/09/09/us-news/ukrainia...
188,35,2025-09-09-17-15-05 +0000,nypost,Jobs growth revised down by staggering 911K — ...,https://nypost.com/2025/09/09/business/jobs-gr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
